In [ ]:
from bluepy import btle
from bluepy.btle import Scanner, DefaultDelegate
import binascii
from time import sleep
import numpy as np

In [ ]:
class BLEDelegate(DefaultDelegate):
    def __init__(self, params):
        DefaultDelegate.__init__(self)
        self.val = [0,1,2,3]

    def handleNotification(self, cHandle, data):
        decimalValue = (binascii.b2a_hex(data))
        splitted = [decimalValue[i:i+4] for i in range(0, len(decimalValue),4)]
        self.val = np.array(list(map(lambda x: int((x),16)/1000, splitted)))
        idx = np.nonzero(self.val)
        self.val = self.val[idx]
        print(self.val)

        
p = btle.Peripheral('28:a7:3c:7a:c6:8c')
p.withDelegate(BLEDelegate(p))

for i in p.getServices():
    print(i.uuid)
    
print("connected")
svc = p.getServiceByUUID('19b10010-e8f2-537e-4f6c-d104768a1214')
char = [i for i in svc.getCharacteristics() if (i.uuid == '19B10012-E8F2-537E-4F6C-D104768A1214')]
handle = char[0].valHandle
p.writeCharacteristic(handle + 1, b'\x01\x00')

while True:
    if p.waitForNotifications(1.0):
        continue